In [ ]:

from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import List, Literal, Optional
import os,json, textwrap, base64

# Optional: set your key here for local testing (avoid committing real keys)
os.environ["OPENAI_API_KEY"] = "" 

client = OpenAI()

MODEL = "gpt-4o-mini"


In [ ]:
from IPython.display import Image, display_html

# Local paths (replace with your generated/saved before & after image paths)
before_img = "shelf_before.png"
after_img = "shelf_after.png"

# ---- Display side-by-side in notebook ----
display_html(f"""
<div style="display:flex; gap:20px;">
  <div><h4>Before</h4><img src="{before_img}" width="300"></div>
  <div><h4>After</h4><img src="{after_img}" width="300"></div>
</div>
""", raw=True)


# ---- Convert both local images to base64 data URLs ----
def to_data_url(path):
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:image/jpeg;base64,{b64}"

before_data_url = to_data_url(before_img)
after_data_url = to_data_url(after_img)

# ---- Send to Vision model for comparison ----
resp = client.chat.completions.create(
    model=MODEL_VISION,
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "Compare the shelves before and after restocking. Highlight changes in 3 concise bullets."},
            {"type": "image_url", "image_url": {"url": before_data_url}},
            {"type": "image_url", "image_url": {"url": after_data_url}}
        ]
    }],
    temperature=0.2
)

print(resp.choices[0].message.content)

In [ ]:
from typing import List, Optional
from pydantic import BaseModel

class ProsCons(BaseModel):
    pros: List[str]
    cons: List[str]
    justifications: List[str]  # 1:1 with pros+cons in order

instruction = f"""
Extract pros and cons from the review.

Rules:
- Detect sarcasm/irony. If a positive-sounding phrase implies a negative, classify as a con.
- A "pro" must be a REAL benefit to a buyer. If none, return an empty pros list.
- Do NOT include joke/ironic "benefits" (e.g., calling a broken phone a good paperweight).
- For every item, include a short justification phrase quoted from the review.
- Return only JSON matching the schema.
Review: {review}
"""

resp = client.chat.completions.parse(
    model=MODEL,
    messages=[{"role":"user","content":instruction}],
    temperature=0,
    response_format=ProsCons
)
data = resp.choices[0].message.parsed


import json

# --- Dummy tool backend ---
def get_weather(city: str) -> str:
    data = {
        "Paris":  "18°C, partly cloudy, light breeze",
        "London": "16°C, light rain, breezy",
    }
    return data.get(city, "Weather data not available")

# --- Tool schema (function calling) ---
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name, e.g., Paris"}
                },
                "required": ["city"]
            }
        }
    }
]

# --- Initial prompt: compare two cities and recommend one for a picnic ---
user_prompt = "Compare Paris vs London weather and recommend one for a picnic."

# First call: let the model decide to call the tool (likely twice: Paris & London)
messages = [{"role": "user", "content": user_prompt}]
first = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)

msg = first.choices[0].message
messages.append(msg)  # keep transcript

# If the model made tool calls, run them and add results to the transcript
if msg.tool_calls:
    for call in msg.tool_calls:
        if call.function.name == "get_weather":
            args = json.loads(call.function.arguments or "{}")
            city = args.get("city", "")
            result = get_weather(city)
            messages.append({
                "role": "tool",
                "tool_call_id": call.id,
                "name": "get_weather",
                "content": result
            })

# Second call: model reads tool outputs and gives a natural-language comparison + recommendation
final = client.chat.completions.create(model=MODEL, messages=messages)
print(final.choices[0].message.content)


# 🧩 Exercises — Function Calling

### Exercise 1: Chained Reasoning — Weather → Plan Recommendation  
Ask:  
> “I’m visiting Paris today. Based on the **current weather**, should I plan an **outdoor run** or go to an **indoor gym**?”  

Do it in two steps:  
1. Call the weather tool to get `temperature` and `windspeed`.
2. Make a **second model call** that applies this policy to the tool output:  
   - If `temperature < 12°C` OR `windspeed ≥ 25 km/h` OR if it's raining→ **INDOOR GYM**  
   - Otherwise → **OUTDOOR RUN**  

Have the model return:  
- **decision** (OUTDOOR RUN / INDOOR GYM)  
- **justification** citing the exact numbers (e.g., “10°C, 28 km/h wind”)  
- a **3-item checklist** (e.g., jacket, water, shoes for gym/run)  


---

### Exercise 2: Multi-Tool Orchestration — Paris Trip Planner  
A traveler asks:  
> “I’m planning a trip to Paris. What’s the weather like, what’s the USD→EUR exchange rate, and can you recommend a book set in Paris to get into the vibe?”  

Tools involved:  
- **Weather API (Open-Meteo)** → helps plan what to wear.  
- **Currency Exchange API (ExchangeRate-API)** → helps plan the budget.  
- **Books API (OpenLibrary)** → suggests a book to set the mood for the journey.  

**Your task:**  
- Combine all three tools.  
- Feed tool results back to the model for a final natural-language answer.  

---

### Exercise 3: Product Price Checker + Inventory Tracker  
A store manager asks:  
> “Do we have Nike Air Max in stock, and what’s the current online price?”  

Tools involved:  
- **Inventory API** → check stock count in local warehouse.  
- **Pricing API (or Web Search)** → fetch competitor/market price.  

**Your task:**  
- Define your own **dummy functions** for both tools (e.g., one dictionary for stock, another for prices).  
- Expose both tools to the model.  
- Ask the scenario question and observe that the model calls *both tools*.  
- Feed the tool results back so the model synthesizes a useful final answer.  

💡 *Extension:* Expand your dummy data with multiple products or warehouses so the query can be richer than just “Nike Air Max.” .  

---

### Exercise 4: Open-Ended Multi-Tool Scenario  
Now it’s your turn.  

- Think of a **real-world scenario** where an assistant would need to combine **at least two tools**.  
- Define which APIs/tools you would connect.  
- Write the natural-language query a user might ask.  
- Sketch how the model would break it down into tool calls and then combine the answers.  

P.S. - If you cannot find a free API for your use-case here, just define a dummy function to return the results. 

